Connect my googleDrive

In [1]:
import os
from google.colab import drive
import numpy as np
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['Colab Notebooks',
 'agg',
 'Batch_Results',
 'resize_images',
 'Flickr',
 'data',
 'resize_images_28',
 'all_image',
 'filename.txt',
 'data_pickle',
 'map_data',
 'data_y.pickle',
 'data.pickle',
 'model.hdf5']

Load data

In [0]:
# load data
import pickle
with open('data.pickle', 'rb') as f:
    X = pickle.load(f)
with open('data_y.pickle', 'rb') as f:
    y = pickle.load(f)

split data to train set vaild set and test set(75% 15% 10%)

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)
X_valid,X_test,y_valid,y_test = train_test_split(X_test,y_test,test_size=0.4)
np.random.seed(200)
np.random.shuffle(X_train)
np.random.seed(200)
np.random.shuffle(X_test)
np.random.seed(200)
np.random.shuffle(y_train)
np.random.seed(200)
np.random.shuffle(y_test)
np.random.seed(200)
np.random.shuffle(X_valid)
np.random.seed(200)
np.random.shuffle(y_valid)

len(X_train),len(X_test),len(X_valid)

(16371, 2184, 3274)

Normalized

In [15]:
x_train = X_train / 255.0
x_test = X_test / 255.0
x_valid = X_valid / 255.0
x_train[0]

array([[[0.02745098, 0.        , 0.02745098],
        [0.01568627, 0.        , 0.01176471],
        [0.        , 0.01176471, 0.        ],
        ...,
        [0.01176471, 0.        , 0.        ],
        [0.00392157, 0.        , 0.01960784],
        [0.        , 0.        , 0.01568627]],

       [[0.03137255, 0.        , 0.01960784],
        [0.        , 0.        , 0.01568627],
        [0.        , 0.00392157, 0.01960784],
        ...,
        [0.00784314, 0.        , 0.01960784],
        [0.00392157, 0.        , 0.02745098],
        [0.00392157, 0.00392157, 0.03529412]],

       [[0.01960784, 0.        , 0.        ],
        [0.        , 0.        , 0.00784314],
        [0.12156863, 0.14117647, 0.16470588],
        ...,
        [0.01960784, 0.00392157, 0.07058824],
        [0.00784314, 0.        , 0.05098039],
        [0.        , 0.        , 0.02745098]],

       ...,

       [[0.00392157, 0.        , 0.02745098],
        [0.00784314, 0.        , 0.05098039],
        [0.        , 0

Fine-tuning xception model

In [16]:
import tensorflow as tf
from tensorflow import keras
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(8, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [17]:
for layer in base_model.layers:
  layer.trainable = False
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(x_train.reshape((x_train.shape[0], 28, 28, 3)), y_train, epochs=5, validation_data=(x_valid.reshape((x_valid.shape[0], 28, 28, 3)),y_valid))

Epoch 1/5
512/512 [==============================] - 79s 154ms/step - loss: 1.9294 - accuracy: 0.2563 - val_loss: 1.9136 - val_accuracy: 0.2676
Epoch 2/5
512/512 [==============================] - 79s 154ms/step - loss: 1.9105 - accuracy: 0.2731 - val_loss: 1.9084 - val_accuracy: 0.2593
Epoch 3/5
512/512 [==============================] - 79s 154ms/step - loss: 1.9055 - accuracy: 0.2749 - val_loss: 1.9033 - val_accuracy: 0.2706
Epoch 4/5
512/512 [==============================] - 82s 160ms/step - loss: 1.9026 - accuracy: 0.2745 - val_loss: 1.9010 - val_accuracy: 0.2883
Epoch 5/5
512/512 [==============================] - 79s 154ms/step - loss: 1.9005 - accuracy: 0.2781 - val_loss: 1.9012 - val_accuracy: 0.2728


In [0]:
# The model can be further trained with base layers unfrozen
for layer in base_model.layers:
  layer.trainable = True

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(x_train.reshape((x_train.shape[0], 28, 28, 3)), y_train, epochs=20, validation_data=(x_valid.reshape((x_valid.shape[0], 28, 28, 3)),y_valid))

Test

In [0]:
model.evaluate(x_test.reshape(x_test.shape[0], 28, 28, 3), y_test)

save

In [0]:
model.save("model.hdf5")

Draw chart

In [0]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.show()


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.show()